In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from importlib import reload 

In [3]:
from deeprank.dataset import DataLoader, PairGenerator, ListGenerator

In [4]:
loader = DataLoader('./config/letor07_mp_fold1.model')

[./data/letor/r5w/word_dict.txt]
	Word dict size: 193367
[./data/letor/r5w/qid_query.txt]
	Data size: 1692
[./data/letor/r5w/docid_doc.txt]
	Data size: 65323
[./data/letor/r5w/embed_wiki-pdc_d50_norm]
	Embedding size: 109282
Generate numpy embed: (193368, 50)


In [5]:
import json
letor_config = json.loads(open('./config/letor07_mp_fold1.model').read())
# device = torch.device("cuda")
device = torch.device("cpu")

In [6]:
Letor07Path = letor_config['data_dir']

letor_config['fill_word'] = loader._PAD_
letor_config['embedding'] = loader.embedding
letor_config['feat_size'] = loader.feat_size
letor_config['vocab_size'] = loader.embedding.shape[0]
letor_config['embed_dim'] = loader.embedding.shape[1]

pair_gen = PairGenerator(rel_file=Letor07Path + '/relation.train.fold%d.txt'%(letor_config['fold']), 
                         config=letor_config)

[./data/letor/r5w/relation.train.fold1.txt]
	Instance size: 47828
Pair Instance Count: 325439


In [7]:
from deeprank import select_module
from deeprank import rank_module

select_module = reload(select_module)
rank_module = reload(rank_module)

In [8]:
select_net = select_module.IdentityNet(config=letor_config)
select_net.train()
select_net = select_net.to(device)

In [9]:
letor_config['simmat_channel'] = 1
letor_config['conv_params'] = [(8, 2, 10)]
letor_config['fc_params'] = [50]
letor_config['dpool_size'] = [3, 10]
letor_config['lr'] = 0.001
letor_config['finetune_embed'] = False
rank_net = rank_module.MatchPyramidNet(config=letor_config)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.train()
optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [10]:
letor_config['simmat_channel'] = 1
letor_config['conv_params'] = [(8, 3, 3)]
letor_config['fc_params'] = [200]
letor_config['dpool_size'] = [3, 10]
letor_config['lr'] = 0.001
letor_config['finetune_embed'] = False
rank_net = rank_module.MatchPyramidNet(config=letor_config)
rank_net = rank_net.to(device)
rank_net.embedding.weight.data.copy_(torch.from_numpy(loader.embedding))
rank_net.train()
optimizer = optim.Adam(rank_net.parameters(), lr=letor_config['lr'])

In [11]:
def to_device(*variables):
    return (variable.to(device) for variable in variables)

In [12]:
import time
start_t = time.time()
for i in range(50):
    X1, X1_len, X2, X2_len, Y, F = pair_gen.get_batch(data1=loader.query_data, data2=loader.doc_data)
    X1, X2, X1_len, X2_len = to_device(X1, X2, X1_len, X2_len)
    output = rank_net(X1, X2, X1_len, X2_len)
    loss = rank_net.pair_loss(output, Y)
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
end_t = time.time()
print('Time Cost: %s s' % (end_t-start_t))

1.012879729270935
0.9917386174201965
0.9694412350654602
0.9858924746513367
0.9554595947265625
0.9368752837181091
0.9394252896308899
0.9478824734687805
0.9384428858757019
0.933110237121582
0.9230427145957947
0.899770200252533
0.9166485667228699
0.9057649970054626
0.8954755663871765
0.8848867416381836
0.8867610096931458
0.895216166973114
0.827060878276825
0.9324040412902832
0.8568477630615234
0.8939464688301086
0.8834852576255798
0.8590696454048157
0.8303807377815247
0.8849024176597595
0.9506853222846985
0.9157118201255798
0.8565244674682617
0.8603119254112244
0.8386087417602539
0.8571120500564575
0.8982592821121216
0.7804421186447144
0.961202085018158
0.8745900988578796
0.8021736145019531
0.8448980450630188
0.8864299654960632
0.8167936205863953
0.842960774898529
0.9015116691589355
0.8842083215713501
0.9195761680603027
0.7860811352729797
0.8499236106872559
0.8554499745368958
0.7465335726737976
0.8227413296699524
0.8032917380332947
Time Cost: 240.31569957733154 s
